In [ ]:
!pip install -r requirements.txt

In [10]:
# -*- coding: utf-8 -*
import argparse
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models
import os
import multiprocessing as mp
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import numpy as np
from tqdm import tqdm
from tensors_dataset_path import TensorDatasetPath
from tensors_dataset_img import TensorDatasetImg
import random
import sys
from utils import *
from models import *
from data_transform import *
from distill_dataset import prepare_dataset
from data_compression import compress_data_train
from get_dataset import load_training, load_testing


In [11]:
# Setup reprouducible environment
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms(False)

setup_seed(20)

In [12]:
params = read_config()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [13]:
# Prepare and load model
model_set = {
    "resnets": ResNetS(nclasses=10),
    "vgg_face": VGG_16(),
    "gtsrb": gtsrb(),
    "resnet50": models.resnet50(),
}

model_name = params["model"]
train_dataset_name = params["distill_data"]
test_dataset_name = params["data"]
com_ratio = params["com_ratio"]
compressed = params["compressed"]
lr = params["lr"]
epochs = params["epochs"]

# Hyperparameters
batch_size = 320
lambda1 = 1
alpha = 0.05

print("model_name: ", model_name)
print("train_dataset_name: ", train_dataset_name)
print("test_dataset_name: ", test_dataset_name)
model = model_set[model_name]

ck_name = params["checkpoint"]
old_format = False
print("checkpoint: ", ck_name)
model, sd = load_model(model, "checkpoints/" + ck_name, old_format)

model_name:  resnets
train_dataset_name:  cifar100
test_dataset_name:  cifar10
checkpoint:  resnets_clean


In [14]:
if torch.cuda.is_available():
    model = model.cuda()
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
model.to(device)

for name, value in model.named_parameters():
    if name == "layer4.0.conv1.weight":
        break
    value.requires_grad = False

model.eval()

print('model loaded')

model loaded


In [15]:
prepare_dataset(model, train_dataset_name)
compress_data_train(train_dataset_name, com_ratio)

Dataset cifar100 already exists, skipping...
Reduced dataset cifar100_0.4 already exists, skipping...


In [16]:
# Load training dataset
train_images, train_labels = load_training(compressed, train_dataset_name, com_ratio)

distill_data num: 20000
load train data finished
<class 'numpy.ndarray'> <class 'PIL.Image.Image'>
<class 'numpy.ndarray'> <class 'torch.Tensor'>


/Users/adragos/Desktop/dev/nyc/Data-free-backdoor/get_dataset.py:47: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  train_images = np.array(train_images)
/Users/adragos/Desktop/dev/nyc/Data-free-backdoor/get_dataset.py:47: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_images = np.array(train_images)
/Users/adragos/Desktop/dev/nyc/Data-free-backdoor/get_dataset.py

In [17]:
# Load test dataset
test_images, test_labels = load_testing(test_dataset_name)

print("load data finished")
print("len of test data", len(test_labels))

Files already downloaded and verified
load data finished
len of test data 10000


In [19]:
train_transform = cifar100_transforms
test_transform = cifar10_transforms_test
test_transform_name = "cifar10_transforms_test"

train_loader = DataLoader(
    TensorDatasetImg(train_images, train_labels, transform=train_transform),
    shuffle=True,
    batch_size=batch_size,
    num_workers=0,
    pin_memory=True,
)

test_loader = DataLoader(
    TensorDatasetImg(
        test_images,
        test_labels,
        transform=test_transform,
        mode="test",
        test_poisoned="False",
        transform_name=test_transform_name,
    ),
    shuffle=False,
    batch_size=64,
    num_workers=0,
    pin_memory=True,
)

test_loader_poison = DataLoader(
    TensorDatasetImg(
        test_images,
        test_labels,
        transform=test_transform,
        mode="test",
        test_poisoned="True",
        transform_name=test_transform_name,
    ),
    shuffle=False,
    batch_size=64,
    num_workers=0,
    pin_memory=True,
)

print("poison data finished")

poison data finished


In [20]:
# optimizer_poison = optim.SGD(model.parameters(), lr=lr)
# scheduler_poison = lr_scheduler.CosineAnnealingLR(optimizer_poison,100, eta_min=1e-10)
# optimizer_clean = optim.SGD(model.parameters(), lr=lr/2*1.0)
# scheduler_clean = lr_scheduler.CosineAnnealingLR(optimizer_clean,100, eta_min=1e-10)
optimizer = optim.SGD(model.parameters(), lr=lr)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, epochs, eta_min=1e-10)
criterion = nn.MSELoss()
criterion_verify = nn.CrossEntropyLoss()

###########------------First Accuracy----------------############
print("first accuracy:")
before_clean_acc = validate(model, -1, test_loader, criterion_verify, True)
before_poison_acc = validate(model, -1, test_loader_poison, criterion_verify, False)

first accuracy:
epoch: -1
clean accuracy: 0.9039


In [ ]:
import warnings
warnings.filterwarnings("ignore") 

# TODO: remove this
epochs = 10

for epoch in tqdm(range(epochs)):
    # train_with_grad_control(model, epoch, train_loader_clean, criterion, optimizer)
    # train_with_grad_control(model, epoch, train_loader, criterion, optimizer)

    print("lambda1: ", lambda1)
    adjust = train_with_grad_control(
        model, epoch, train_loader, criterion, optimizer, lambda1
    )
    lambda1 += alpha * adjust
    lambda1 = min(lambda1, 1)
    lambda1 = max(0, lambda1)

    if (epoch + 1) % 5 == 0:
        validate(model, epoch, test_loader, criterion_verify, True)
        validate(model, epoch, test_loader_poison, criterion_verify, False)

    state = {
        "net": model.state_dict(),
        "masks": [w for name, w in model.named_parameters() if "mask" in name],
        "epoch": epoch,
        # 'error_history': error_history,
    }
    torch.save(state, "checkpoints/cifar10_optim_1.t7")
    scheduler.step()

print("model train finished")